# 1 Definição da Constelação de Fatos

**Tabelas de dimensão**

- data (dataPK, dataCompleta, dataDia, dataMes, dataSemestre, dataAno)
- localidade (localidadePk, latitude, longitude, cidade, estado, região, país)
- tipo-cancer (tipoCancerPK, tipoCancer, mortalidade, taxaIncidenciTotal)
- faixa-etaria (faixaPK, faixaIdade, idadeMin, idadeMax)
- sexo (sexoPK, sexo)

**Tabelas de fatos**
- cancer (dataPK, localidadePK, tipoCancerPK, sexoPK, faixaPK, casoConfirmados, obitosConfirmados)
- clima (dataPK, localidadePK, temperaturaMedia, temperaturaMax, temperaturaMin, radiacaoUV, radiacaoUVA, radiacaoUVB, precipitacao)

In [52]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, expr, to_date
from pyspark.sql import functions as F

# Python 3.11
# Java 11
# PySpark == 3.3.2

In [53]:
spark = SparkSession.builder \
    .appName("OLAP - P2") \
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-aws:3.3.2,"
            "com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

print(spark.version)

3.3.2


## Extract

In [54]:
df_cancer1 = spark.read.csv("s3a://datalake/cancer-1.csv", header=True, inferSchema=True)
df_cancer2 = spark.read.csv("s3a://datalake/cancer-2.csv", header=True, inferSchema=True)
df_cancer = df_cancer1.union(df_cancer2)

print("Numero de Tuplas: ", df_cancer.count())
df_cancer.show()

Numero de Tuplas:  789264
+----------+------------+-----------+-------------+------+---------+------+-----------+--------+--------------------+---------+-----------+----+--------------------+--------------------+--------------------+
|measure_id|measure_name|location_id|location_name|sex_id| sex_name|age_id|   age_name|cause_id|          cause_name|metric_id|metric_name|year|                 val|               upper|               lower|
+----------+------------+-----------+-------------+------+---------+------+-----------+--------+--------------------+---------+-----------+----+--------------------+--------------------+--------------------+
|         1|      Óbitos|       4750|         Acre|     1|Masculino|     8|15 -19 anos|     459|Melanoma maligno ...|        1|     Número|2001|0.008899466563112322|0.010069127561452958|0.007914220987606399|
|         1|      Óbitos|       4750|         Acre|     2| Feminino|     8|15 -19 anos|     459|Melanoma maligno ...|        1|     Número|200

In [55]:
df_clima = spark.read.csv("s3a://datalake/climate.csv", header=True, inferSchema=True)

print("Numero de Tuplas: ", df_clima.count())
df_clima.show()

Numero de Tuplas:  42721900
+---------------+--------+---------+----+---+-----+-------+-------+-------------------+--------------+--------------+-----------+
|         cidade|latitude|longitude|YEAR|DOY|  T2M|T2M_MAX|T2M_MIN|ALLSKY_SFC_UV_INDEX|ALLSKY_SFC_UVA|ALLSKY_SFC_UVB|PRECTOTCORR|
+---------------+--------+---------+----+---+-----+-------+-------+-------------------+--------------+--------------+-----------+
|Abadia de Goiás|-16.7573| -49.4412|2001|  1|23.12|  26.25|  20.85|               2.29|           1.2|          0.04|      10.58|
|Abadia de Goiás|-16.7573| -49.4412|2001|  2|21.86|  24.46|  19.76|               1.95|          1.05|          0.03|        4.7|
|Abadia de Goiás|-16.7573| -49.4412|2001|  3|21.72|  25.39|  18.91|               2.27|           1.2|          0.04|        4.3|
|Abadia de Goiás|-16.7573| -49.4412|2001|  4|23.14|  27.84|   18.3|               1.81|          0.95|          0.03|       1.72|
|Abadia de Goiás|-16.7573| -49.4412|2001|  5|23.18|  29.38|  1

## Transform

In [ ]:
df_data = df_clima.withColumn(
    "dataCompleta",
    expr("date_add(to_date(concat(YEAR, '-01-01')), DOY - 1)")
).select(
    "dataCompleta",
    F.dayofmonth("dataCompleta").alias("dataDia"),
    F.month("dataCompleta").alias("dataMes"),
    ((F.month("dataCompleta")-1)/ 6).cast("int").alias("dataSemestre"),
    F.year("dataCompleta").alias("dataAno")
).distinct()

# criar chave artificial
df_data = df_data.withColumn("dataPK", F.monotonically_increasing_id())

df_data = df_data.select("dataPK", "dataCompleta", "dataDia", "dataMes", "dataSemestre", "dataAno")

print("Numero de Tuplas: ", df_data.count())
df_data.show()

Numero de Tuplas:  7670


+------------+-------+-------+------------+-------+------+
|dataCompleta|dataDia|dataMes|dataSemestre|dataAno|dataPK|
+------------+-------+-------+------------+-------+------+
|  2001-07-27|     27|      7|           1|   2001|     0|
|  2001-10-27|     27|     10|           1|   2001|     1|
|  2002-02-08|      8|      2|           0|   2002|     2|
|  2002-03-06|      6|      3|           0|   2002|     3|
|  2002-06-15|     15|      6|           0|   2002|     4|
|  2002-08-11|     11|      8|           1|   2002|     5|
|  2002-09-08|      8|      9|           1|   2002|     6|
|  2003-03-30|     30|      3|           0|   2003|     7|
|  2003-07-07|      7|      7|           1|   2003|     8|
|  2003-08-23|     23|      8|           1|   2003|     9|
|  2004-03-09|      9|      3|           0|   2004|    10|
|  2005-12-10|     10|     12|           1|   2005|    11|
|  2006-02-14|     14|      2|           0|   2006|    12|
|  2006-08-24|     24|      8|           1|   2006|    1

In [75]:
df_localidade = (
    df_clima.select("cidade", "latitude", "longitude")
            .distinct()
            .join(
                spark.read.csv("s3a://datalake/estado-bounding.csv", header=True, inferSchema=True),
                (F.col("latitude").between(F.col("lat_min"), F.col("lat_max"))) &
                (F.col("longitude").between(F.col("lon_min"), F.col("lon_max"))),
                how="left"
            )
            .withColumn("localidadePK", F.monotonically_increasing_id())
            .select("localidadePK", "latitude", "longitude", "cidade", "estado", "regiao", "pais")
)

print("Numero de Tuplas: ", df_localidade.count())
df_localidade.show(truncate=False)

Numero de Tuplas:  9339


+------------+--------+---------+--------------------+-------------------+--------+------+
|localidadePK|latitude|longitude|cidade              |estado             |regiao  |pais  |
+------------+--------+---------+--------------------+-------------------+--------+------+
|0           |-12.1335|-38.4208 |Alagoinhas          |Bahia              |Nordeste|Brasil|
|1           |-11.5706|-47.1792 |Almas               |Tocantins          |Norte   |Brasil|
|2           |-8.7061 |-35.5263 |Água Preta          |Pernambuco         |Nordeste|Brasil|
|3           |-6.9466 |-35.5332 |Alagoinha           |Paraíba            |Nordeste|Brasil|
|4           |-6.9466 |-35.5332 |Alagoinha           |Rio Grande do Norte|Nordeste|Brasil|
|5           |-27.4333|-51.9044 |Alto Bela Vista     |Rio Grande do Sul  |Sul     |Brasil|
|6           |-27.4333|-51.9044 |Alto Bela Vista     |Santa Catarina     |Sul     |Brasil|
|7           |-13.2488|-41.6619 |Abaíra              |Bahia              |Nordeste|Brasil|

In [ ]:
df_sexo = df_cancer.select(
    F.col("sex_name").alias("sexo")
).distinct()

df_sexo = df_sexo.withColumn("sexoPK", F.monotonically_increasing_id())

print("Numero de Tuplas: ", df_sexo.count())
df_sexo.show()

Numero de Tuplas:  2


+---------+------+
|     sexo|sexoPK|
+---------+------+
| Feminino|     0|
|Masculino|     1|
+---------+------+



In [ ]:
total_casos = df_cancer.filter(F.col("metric_name") == "Número") \
    .agg(F.sum("val").alias("total_casos_geral")) \
    .collect()[0]["total_casos_geral"]

df_tipo_cancer = (
    df_cancer.filter(F.col("metric_name") == "Número") 
            .groupBy("cause_name")
            .agg(
                F.sum(F.when(F.col("measure_name") == "Óbitos", F.col("val")).otherwise(0)).alias("total_obitos"),
                F.sum(F.when(F.col("measure_name") != "Óbitos", F.col("val")).otherwise(0)).alias("total_casos")
            ) 
            .withColumn(
                "mortalidade",
                F.round(F.when(F.col("total_casos") > 0, (F.col("total_obitos") / F.col("total_casos")) * 100).otherwise(0),2)
            ) 
            .withColumn(
                "taxaIncidenciaTotal",
                F.round((F.col("total_casos") / F.lit(total_casos)) * 100, 2)
            ) 
            .withColumn(
                "tipoCancerPK",
                F.monotonically_increasing_id()
            ) 
            .select(
                "tipoCancerPK",
                F.col("cause_name").alias("tipoCancer"),
                "mortalidade",
                "taxaIncidenciaTotal"
            )
)

print("Numero de Tuplas: ", df_tipo_cancer.count())
df_tipo_cancer.show(truncate=False)

Numero de Tuplas:  3


+------------+------------------------------------------------------------+-----------+-------------------+
|tipoCancerPK|tipoCancer                                                  |mortalidade|taxaIncidenciaTotal|
+------------+------------------------------------------------------------+-----------+-------------------+
|0           |Câncer de pele não melanoma (carcinoma basocelular)         |0.0        |89.55              |
|1           |Melanoma maligno da pele                                    |35.94      |3.52               |
|2           |Câncer de pele não melanoma (carcinoma de células escamosas)|23.84      |4.57               |
+------------+------------------------------------------------------------+-----------+-------------------+



In [ ]:
df_faixa_etaria = (
    df_cancer.select(F.col("age_name").alias("faixaEtaria"))
            .distinct()
            .withColumn(
                "faixaEtaria",
                F.regexp_replace(F.regexp_replace(F.col("faixaEtaria"), "anos", ""), " ", "")
            )
            .withColumn(
                "idadeMin",
                F.when(
                    F.col("faixaEtaria").contains("+"),
                    F.regexp_replace(F.col("faixaEtaria"), "[^0-9]", "").cast("int")
                ).otherwise(
                    F.split(F.col("faixaEtaria"), "-")[0].cast("int")
                )
            )
            .withColumn(
                "idadeMax",
                F.when(
                    F.col("faixaEtaria").contains("+"),
                    F.lit(130) 
                ).otherwise(
                    F.split(F.col("faixaEtaria"), "-")[1].cast("int")
                )
            )
            .withColumn("faixaPK", F.monotonically_increasing_id())
            .select("faixaPK", "faixaEtaria", "idadeMin", "idadeMax")
)

print("Numero de Tuplas: ", df_faixa_etaria.count())
df_faixa_etaria.show()

Numero de Tuplas:  30


+-------+-----------+--------+--------+
|faixaPK|faixaEtaria|idadeMin|idadeMax|
+-------+-----------+--------+--------+
|      0|      20-54|      20|      54|
|      1|      35-39|      35|      39|
|      2|      75-84|      75|      84|
|      3|        95+|      95|     130|
|      4|      85-89|      85|      89|
|      5|      10-14|      10|      14|
|      6|      15-19|      15|      19|
|      7|      80-84|      80|      84|
|      8|      45-49|      45|      49|
|      9|      15-49|      15|      49|
|     10|      50-69|      50|      69|
|     11|      40-44|      40|      44|
|     12|      20-24|      20|      24|
|     13|      10-54|      10|      54|
|     14|      10-24|      10|      24|
|     15|      75-79|      75|      79|
|     16|      50-54|      50|      54|
|     17|      90-94|      90|      94|
|     18|        5-9|       5|       9|
|     19|      60-64|      60|      64|
+-------+-----------+--------+--------+
only showing top 20 rows

